## ***Import Libraries***

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 16.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import cv2
from ultralytics import YOLO
import torch
import warnings
warnings.filterwarnings('ignore')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 59.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 88.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, w

In [4]:
import mlflow
import mlflow.tensorflow  # Optional if logging TensorFlow/Keras models

# Local tracking URI (logs to /kaggle/working/mlruns)
mlflow.set_tracking_uri("file:/kaggle/working/mlruns")

# Alternatively, if using a remote server, set it like this:
# mlflow.set_tracking_uri("http://your-mlflow-server-url")

# Set experiment name for tracking
mlflow.set_experiment("emotion-detection")


2024/11/13 19:49:28 INFO mlflow.tracking.fluent: Experiment with name 'emotion-detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///kaggle/working/mlruns/123644158422531315', creation_time=1731527368465, experiment_id='123644158422531315', last_update_time=1731527368465, lifecycle_stage='active', name='emotion-detection', tags={}>

## ***YOLO Training***

In [5]:
yaml_path = "/kaggle/input/yolo-balanced-dataset/data.yaml"

In [6]:
class YOLOTrainer:
    def __init__(self, task='', mode='', model='', imgsz=640, data='',
                 device=None, epochs=None, batch=None, learning_rate=None, optimizer=None, weight_decay=None, name='', exist_ok=True):
        self.task = task
        self.mode = mode
        self.model = model
        self.imgsz = imgsz
        self.data = data
        self.device = device
        self.epochs = epochs
        self.batch = batch
        self.name = name
        self.exist_ok = exist_ok
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.weight_decay = weight_decay

    def load_model(self):
        model = YOLO(self.model)
        return model

    def train(self):
        # Start an MLflow run to log parameters and metrics
        with mlflow.start_run(run_name=self.name):
            # Log hyperparameters
            mlflow.log_param("task", self.task)
            mlflow.log_param("mode", self.mode)
            mlflow.log_param("model", self.model)
            mlflow.log_param("imgsz", self.imgsz)
            mlflow.log_param("data", self.data)
            mlflow.log_param("device", self.device)
            mlflow.log_param("epochs", self.epochs)
            mlflow.log_param("batch", self.batch)
            mlflow.log_param("learning_rate", self.learning_rate)
            mlflow.log_param("optimizer", self.optimizer)
            mlflow.log_param("weight_decay", self.weight_decay)
            
            # Train the model
            model = self.load_model()
            results = model.train(
                task=self.task,
                mode=self.mode,
                data=self.data,
                device=self.device,
                epochs=self.epochs,
                batch=self.batch,
                imgsz=self.imgsz,
                name=self.name,
                exist_ok=self.exist_ok,
                lr0=self.learning_rate,
                optimizer=self.optimizer,
                weight_decay=self.weight_decay
            )
            
            # Log metrics from the training process
            mlflow.log_metric("train_loss", results.get('train_loss'))
            mlflow.log_metric("val_loss", results.get('val_loss'))
            mlflow.log_metric("accuracy", results.get('accuracy'))
    
            # Return the results for further use
            return results


In [7]:
import time

def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)

    try:
        # End any active MLflow run before starting a new one
        if mlflow.active_run():
            print(f"Ending active run with UUID: {mlflow.active_run().info.run_id}")
            mlflow.end_run()
        else:
            print("GOOD")

    except Exception as e:
        print(f"Error ending active run: {e}")

    # Start a new MLflow run for this trial
    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        try:
            # Log hyperparameters
            mlflow.log_params({
                'learning_rate': learning_rate,
                'batch_size': batch_size,
                'weight_decay': weight_decay
            })

            # Initialize trainer with sampled hyperparameters
            trainer = YOLOTrainer(
                task='detect',
                mode='train',
                model='yolo11m.pt',
                imgsz=640,
                data=yaml_path,
                device=0,
                epochs=40,
                batch=batch_size,
                learning_rate=learning_rate,
                optimizer='Adam',
                weight_decay=weight_decay,
                name='emotion-detection',
                exist_ok=True
            )

            # Train and log results
            results = trainer.train()
            val_loss = results.get('val_loss')
            mlflow.log_metric("val_loss", val_loss)

        except Exception as e:
            print(f"Error during trial {trial.number}: {e}")
            return float('inf')  # Return a high value to indicate failure

    return val_loss


In [9]:
!pip install optuna

In [12]:
import optuna

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
print("Best parameters:", study.best_params)

[I 2024-11-13 19:53:04,330] A new study created in memory with name: no-name-3b298075-36d6-45c7-a9b0-f8105e6e5293
[I 2024-11-13 19:53:04,410] Trial 0 finished with value: inf and parameters: {'learning_rate': 0.0012574978117087187, 'batch_size': 32, 'weight_decay': 1.3483208521731968e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,425] Trial 1 finished with value: inf and parameters: {'learning_rate': 0.0004221697501266341, 'batch_size': 32, 'weight_decay': 2.0796320511827808e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,439] Trial 2 finished with value: inf and parameters: {'learning_rate': 0.003006200412339819, 'batch_size': 32, 'weight_decay': 0.006511391908589099}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,456] Trial 3 finished with value: inf and parameters: {'learning_rate': 0.00018035812718167623, 'batch_size': 8, 'weight_decay': 0.00011183442500059742}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,471] Trial 4 finished with

GOOD
Error during trial 0: Run with UUID 3f189badc11940acb48a93ecbfff3a96 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 1: Run with UUID 9148371b423c4513957712eddfcbbfc0 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 2: Run with UUID 0ea7c253bfc349a886d38bcc702337dc is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 3: Run with UUID deeefc1703524a439ce2c02c087fffce is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 4: Run with UUID df8935664a17473d8b8d769a4daf2faa is already active. To start a new run, first end the curre

[I 2024-11-13 19:53:04,546] Trial 9 finished with value: inf and parameters: {'learning_rate': 0.0006073518956381393, 'batch_size': 16, 'weight_decay': 0.0005068350143147086}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,578] Trial 10 finished with value: inf and parameters: {'learning_rate': 0.0013959835025784471, 'batch_size': 32, 'weight_decay': 0.002041988146386122}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,605] Trial 11 finished with value: inf and parameters: {'learning_rate': 0.000640710530019083, 'batch_size': 32, 'weight_decay': 1.1245235917621915e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,631] Trial 12 finished with value: inf and parameters: {'learning_rate': 0.0004412719821579298, 'batch_size': 32, 'weight_decay': 3.4719185165812185e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,658] Trial 13 finished with value: inf and parameters: {'learning_rate': 0.00011135731646360803, 'batch_size': 32, 'weight_decay': 3.03244

Error during trial 9: Run with UUID 2bec4a2bbb7d4afeb86b7610e508d87f is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 10: Run with UUID efe147ef833b4c78a5d1085d585b6552 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 11: Run with UUID 75b87fbfd5e848659ed2a71e965d7d70 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 12: Run with UUID e9370a05d04e404bbbeaa6720a5c9332 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 13: Run with UUID 5e0ecf22ae8e46a2917c446cd72df421 is already active. To start a new run, first end the curren

[I 2024-11-13 19:53:04,757] Trial 16 finished with value: inf and parameters: {'learning_rate': 0.0009224734629918075, 'batch_size': 32, 'weight_decay': 6.170488195466058e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,785] Trial 17 finished with value: inf and parameters: {'learning_rate': 0.00029704725161430767, 'batch_size': 32, 'weight_decay': 1.0519066675499152e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,811] Trial 18 finished with value: inf and parameters: {'learning_rate': 0.0021704439351159953, 'batch_size': 16, 'weight_decay': 6.127843537883223e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,838] Trial 19 finished with value: inf and parameters: {'learning_rate': 0.0008211849579865925, 'batch_size': 8, 'weight_decay': 0.001225289069823691}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:04,865] Trial 20 finished with value: inf and parameters: {'learning_rate': 0.0004900771491898465, 'batch_size': 32, 'weight_decay': 2.53087

...

Error during trial 31: Run with UUID 9060e7e896e34b8b84c1361c1fa47240 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 32: Run with UUID cb8eac745c2041a19e7e5cdaa3ea508e is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 33: Run with UUID 11c25efba5b04c45bf8a05d597d268e8 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 34: Run with UUID fe8cc51decb346329612546dfc690215 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 35: Run with UUID b34b0cabb2094d71acb5fec8c2125579 is already active. To start a new run, first end the curre

[I 2024-11-13 19:53:05,392] Trial 39 finished with value: inf and parameters: {'learning_rate': 0.0005513005482378618, 'batch_size': 16, 'weight_decay': 0.0003905394082802015}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,422] Trial 40 finished with value: inf and parameters: {'learning_rate': 0.0003162808100582355, 'batch_size': 32, 'weight_decay': 1.840237179047896e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,452] Trial 41 finished with value: inf and parameters: {'learning_rate': 0.008520841629765564, 'batch_size': 8, 'weight_decay': 0.00024989504441996936}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,481] Trial 42 finished with value: inf and parameters: {'learning_rate': 0.005231496268799028, 'batch_size': 8, 'weight_decay': 8.649063562115392e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,510] Trial 43 finished with value: inf and parameters: {'learning_rate': 0.001461385826610746, 'batch_size': 8, 'weight_decay': 0.0001418531

GOOD
Error during trial 39: Run with UUID 4b911045c7dd4066a60ccf1a95bd5aa3 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 40: Run with UUID 137f2ee176384490b5c24b9c9eba0ee3 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 41: Run with UUID 640dc37093714484b50bf066c95188c7 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 42: Run with UUID 0dc3343cd8114a6b8cf0452dd02f4410 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 43: Run with UUID a259e65d366046dcad783593a5d76b96 is already active. To start a new run, first end the 

[I 2024-11-13 19:53:05,595] Trial 46 finished with value: inf and parameters: {'learning_rate': 0.002559596214767682, 'batch_size': 32, 'weight_decay': 2.8234538822116192e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,623] Trial 47 finished with value: inf and parameters: {'learning_rate': 0.0003965053998463759, 'batch_size': 16, 'weight_decay': 4.570757518929721e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,651] Trial 48 finished with value: inf and parameters: {'learning_rate': 0.0002301368807058238, 'batch_size': 8, 'weight_decay': 1.100235277555698e-05}. Best is trial 0 with value: inf.
[I 2024-11-13 19:53:05,680] Trial 49 finished with value: inf and parameters: {'learning_rate': 0.0006756301100922385, 'batch_size': 32, 'weight_decay': 0.001902024001549024}. Best is trial 0 with value: inf.


GOOD
Error during trial 46: Run with UUID af4417ef111f468ebd8ccb675bdb196e is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 47: Run with UUID 7bf879f9a69140d982d8955f213582d9 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 48: Run with UUID 8541a343e0524db5b3e2c0937596c631 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
GOOD
Error during trial 49: Run with UUID bf1794e999524cd79589ed1397bd5a99 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True
Best parameters: {'learning_rate': 0.0012574978117087187, 'batch_size': 32, 'weight_decay': 1.3483208521731968e-05}


In [ ]:
# After the study optimization
best_params = study.best_params
print("Best parameters:", best_params)

# Now retrain the model with these best parameters
trainer = YOLOTrainer(
    task='detect',
    mode='train',
    model='yolo11m.pt',
    imgsz=640,
    data=yaml_path,  # Make sure this is correctly defined
    device=0,
    epochs=40,
    batch=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    optimizer='Adam',
    weight_decay=best_params['weight_decay'],
    name='emotion-detection',
    exist_ok=True
)

# Start training with the best parameters
trainer.train()
